El objetivo de esta notebook es poder predecir a partir de informacion contenida en datasets de IMDB (https://www.imdb.com) si una pelicula en produccion va a ser bien recibida o mal recibida por el publico.

Importamos las principales librerias:

In [ ]:
import pandas as pd
import numpy as np

Definimos los datasets que vamos a utilizar:

In [ ]:
# local
data_location = "./data_name_basics.tsv"
data_location1 = "./data_title_akas.tsv"
data_location2 = "./data_title_basics.tsv"
data_location3 = "./data_title_crew.tsv"
data_location4 = "./data_title_principals.tsv"
data_location5 = "./data_title_ratings.tsv"



El primer dataset contiene informacion sobre los directores:

In [ ]:
tsv_file = open(data_location, encoding="UTF8")
data = pd.read_csv(tsv_file, delimiter="\t", low_memory=False)
data.head(3)

In [ ]:
name_basics = data

El segundo dataset contiene informacion sobre todos los distintos titulos en imdb:

In [ ]:
tsv_file = open(data_location2, encoding="UTF8")
data2 = pd.read_csv(tsv_file, delimiter="\t", low_memory=False)
data2.head(3)

El tercer dataset muestra el ID de cada pelicula junto con el ID su respectivo director:

In [ ]:
tsv_file = open(data_location3, encoding="UTF8")
data3 = pd.read_csv(tsv_file, delimiter="\t", low_memory=False)
data3.head(3)

El cuarto dataset muestra informacion sobre ID's de directores, productores, etc., asociados al ID de cada pelicula:

In [ ]:
tsv_file = open(data_location4, encoding="UTF8")
data5 = pd.read_csv(tsv_file, delimiter="\t", low_memory=False)
data5.head(3)

El quinto dataset muestra para el ID de cada pelicula, su rating y el numero de votos:

In [ ]:
tsv_file = open(data_location5, encoding="UTF8")
data6 = pd.read_csv(tsv_file, delimiter="\t", low_memory=False)
data6.head(3)

Ahora vamos a llamar a nuestro segundo dataset que contiene informacion sobre los titulos:

In [ ]:
data2.head(20)

Vemos el numero de columnas y filas:

In [ ]:
data2.shape

Vemos el tipo de dato de cada columna:

In [ ]:
data2.dtypes

Vemos los valores de la columna "titleType":

In [ ]:
data2.titleType.value_counts()

Como hay muchos tipos de titulos distintos, y a nosotros solo nos interesan las peliculas, vamos a crear una mascara para poder filtrar por peliculas y la aplicamos:

In [ ]:
mask_movie = data2.titleType == 'movie'
df_m = data2[mask_movie]

Corroboramos que el numero de filas haya disminuido:

In [ ]:
df_m.shape

Vemos que valores tiene la columna "startYear":

In [ ]:
df_m["startYear"].value_counts()

Notamos que la columna "startYear" tiene como valor "\N", por lo cual vamos a crear y aplicar una mascara para poder filtrar esos casos:

In [ ]:
mask_year = df_m["startYear"] != "\\N"
df_sin_n = df_m[mask_year]

Verificamos que esas filas ya no se encuentren en el nuevo dataframe:

In [ ]:
df_sin_n["startYear"].value_counts()

Convertimos a int a la columna y verificamos:

In [ ]:
df_sin_n["startYear"] = df_sin_n["startYear"].astype(int) 

In [ ]:
df_sin_n["startYear"].dtypes

Calculamos la cantidad de columnas y filas de este dataframe actualizado:

In [ ]:
df_sin_n.shape

Como el objetivo trata a cerca de peliculas en produccion actualmente, solo nos interesa quedarnos con inputs que no sean demasiado antiguos, por lo cual solo nos vamos a limitar a trabajar con peliculas creadas a partir de 1960. Para eso vamos a crear una mascara y aplicarla al dataframe:

In [ ]:
mask_year = df_sin_n["startYear"] > 1959
df_sin_n_1960 = df_sin_n[mask_year]

Verificamos que el numero de filas haya disminuido y hacemos un display de este nuevo dataframe:

In [ ]:
df_sin_n_1960.shape


In [ ]:
df_sin_n_1960.head(20)


Ahora dropeamos la columna "endYear" ya que no aporta informacion util y llamamos al dataframe nuevamente:

In [ ]:
df_sin_n_1960 = df_sin_n_1960.drop(axis=1, columns='endYear')

In [ ]:
df_sin_n_1960.head(20)

Creamos y aplicamos una mascara que permita filtrar en todas las columnas a las celdas que valgan "\N":

In [ ]:
mask_N = df_sin_n_1960 != "\\N"
df_sin_n_1960_N = df_sin_n_1960[mask_N]

Ahora creamos un loop que para convertir a los valores "\N" en NaN y llamamos al dataframe:

In [ ]:
for columns in df_sin_n_1960_N.columns:
    for f in columns:
        if f == "\\N":
            f = np.NaN

In [ ]:
df_sin_n_1960_N.head(20)

Vemos el procentaje de valores nulos por columna en el dataframe y el tipo de dato de cada columna:

In [ ]:
nulos = df_sin_n_1960_N.isnull().sum()/df_sin_n_1960_N.shape[0]*100
nulos

In [ ]:
df_sin_n_1960_N.dtypes

Creamos un nuevo dataframe donde estan solo los valores no nulos y tambien verificamos que haya disminuido el numero de filas:

In [ ]:
data_filas_completas = df_sin_n_1960_N.dropna()
print(df_sin_n_1960_N.shape)
print(data_filas_completas.shape)

In [ ]:
data_filas_completas.isna().sum()

Renombramos este ultimo dataframe y lo llamamos "title_basics":

In [ ]:
title_basics = data_filas_completas

Ahora crearemos un indice que utilizaremos mas adelante a partir de la columna "tconst" de este dataframe (el ID de cada pelicula):

In [ ]:
indice = title_basics["tconst"]#Ω

In [ ]:
indice

Ahora creamos el dataframe "ratings" a partir del dataframe que posee datos sobre el puntaje de cada pelicula, y vemos cuantas columnas y filas tiene:

In [ ]:
ratings = data6
ratings.shape

In [ ]:
ratings.head(20)

Creamos un nuevo dataframe "ratings_indice" que surge de la combinacion entre "ratings" e "indice" y donde como maximo solo van a aparecer los ratings de las peliculas que quedaron luego de nuestros filtros anteriores:

In [ ]:
ratings_indice = pd.merge(ratings, indice, how="inner", on=["tconst"])

Vemos el tamaño de este dataframe y lo llamamos:

In [ ]:
ratings_indice.shape

In [ ]:
ratings_indice

Ahora vamos a llamar a otros datasets que contienen informacion sobre los directores que trabajaron en las peliculas y datos en detalle acerca de cada una de ellas:

In [ ]:
crew = data3
principals = data5

In [ ]:
crew.head()

In [ ]:
principals.head()

Vemos cuales son los valores de la columna "category" en dataset sobre las peliculas:

In [ ]:
principals["category"].value_counts()

Teniendo en cuenta la importancia de la direccion en las peliculas y dado el gran numero de actores y actrices en el reparto, solo propondremos trabajar con los directores de cada pelicula. Para eso vamos a crear una mascara y la vamos a aplicar:

In [ ]:
directores_mask = (principals["category"] == "director")
staff = principals[directores_mask]

Verificamos que el numero de filas haya disminuido:

In [ ]:
print(principals.shape)
print(staff.shape)

Creamos un nuevo dataframe "staff_indice" que surge de la combinacion entre "staff" e "indice":

In [ ]:
staff_indice = pd.merge(staff, indice, how="inner", on=["tconst"])

Como hay columnas que no aportan informacion util y/o tienen NaN's vamos a eliminarlas y volveremos a llamar al dataframe:

In [ ]:
staff_indice = staff_indice.drop(columns=["job","characters","ordering"])

In [ ]:
staff_indice.shape

In [ ]:
staff_indice.head()

Ahora hacemos un merge entre el dataframe que habiamos creado inicialmente "title_basics", y "ratings_indice":

In [ ]:
title_ratings = pd.merge(title_basics, ratings_indice, on='tconst')


Verificamos que el rating aparezca en este nuevo dataframe:

In [ ]:
title_ratings

Ahora dropeamos la columna 'category':

In [ ]:
staff_sinc = staff_indice.drop(['category'], axis=1)

In [ ]:
staff_sinc

In [ ]:
mask_staff_sind = staff_sinc['tconst'].drop_duplicates(keep= "first")


In [ ]:
mask_staff_sind

Ahora realizamos un merge entre los dataframes staff_sinc y title_ratings a partir de 'tconst':

In [ ]:
imdb =title_ratings.merge(staff_sinc, on = 'tconst', validate = "many_to_many")
imdb 


In [ ]:
crew.head(3)

Arreglamos el dataframe para poder trabajar con el mas adelante:

In [ ]:
imdb ['tconst'].unique().shape

In [ ]:
strip = imdb .genres.apply(lambda x: x.replace(' ', ''))

In [ ]:
imdb.genres = strip

In [ ]:
imdb 

Para poder continuar y modelar vamos a crear dummies a partir de la columna genero:

In [ ]:
imdb_dum = imdb.genres.str.join(sep='').str.get_dummies(sep=',')

In [ ]:
imdb_dum

Unimos este ultimo dataframe con el dataframe anterior, imdb:

In [ ]:
imdb_condumgen =imdb.join(imdb_dum)
imdb_condumgen

In [ ]:
imdb_condumgen["nconst"]

Vamos a droppear las peliculas que se encuentran repetidas ya que solo nos vamos a concentrar en utilizar como input al director principal y no al secundario para asi evitar trabajar con demasiadas variables en exceso:

In [ ]:
imdb_2=imdb_condumgen.drop_duplicates(keep='first', subset='tconst')
imdb_2.shape

In [ ]:
nconst_filtro = imdb_2["nconst"]

Ahora realizamos un merge entre el dataframe anterior y el dataset que contiene informacion acerca de los directores:

In [ ]:
imdb_2 = imdb_2.merge(name_basics,how="inner",on=["nconst"])


In [ ]:
imdb_2.columns

In [ ]:
name_basics.head(3)

Para poder realizar un analisis no sesgado vamos a utilizar como aceptable a los ratings (que serviran para definir a nuestra variable a predecir) que tengan al menos 1.000 votos por el publico. Para eso creamos una mascara y la aplicamos:

In [ ]:
imdb_2["numVotes"].value_counts(normalize=True)

In [ ]:
imdb_2["numVotes"].mean()

In [ ]:
mask_votos = imdb_2.numVotes > 1000

In [ ]:
imdb_mas50 = imdb_2[mask_votos]

In [ ]:
imdb_mas50 = imdb_mas50.drop(["tconst", "titleType","primaryTitle","originalTitle", "genres"], axis= 1)

In [ ]:
for n in imdb_mas50['averageRating']:
    n=int(n)
       

In [ ]:
imdb_mas50['averageRating']

In [ ]:
imdb_mas50['averageRating']

In [ ]:
imdb_mas50['averageRating'].unique()

Definimos las categorias que utilizaremos de la siguiente manera: si el rating es menor o igual a 6 entonces sera una pelicula mala, si es superior entonces sera buena. Originalmente la intencion era crear mas escenarios para las clasificaciones, pero debido al procesamiento que requeria la notebook para procesar, debimos disminuir las categorias:

In [ ]:
Clasificacion=[]

for x in imdb_mas50['averageRating']:
    if x<=6:
        Clasificacion.append(0)
    else:
        Clasificacion.append(1)
#     if x<=2:
#         Clasificacion.append(0)
#     elif x<=4:
#         Clasificacion.append(1)
#     elif x<=6:
#         Clasificacion.append(2)
#     elif x<=8:
#         Clasificacion.append(3)
#     elif x<=10:
#         Clasificacion.append(4)


In [ ]:
Clasificacion=pd.Series(Clasificacion)
Clasificacion

In [ ]:
imdb_mas50['Clasificacion']=Clasificacion.values

In [ ]:
imdb_mas50.columns

In [ ]:
print(imdb_mas50[['averageRating', 'Clasificacion']])

In [ ]:
imdb_mas50['Clasificacion'].isnull().sum()

Dropeamos columnas porque no tienen ningun registro o no tienen utilidad en adelante:

In [ ]:

imdb_mas50 = imdb_mas50.drop(columns = ["Game-Show","Reality-TV","Talk-Show"])


In [ ]:
imdb_mas50.columns

In [ ]:
imdb_mas50 = imdb_mas50.drop(columns=['numVotes', 'birthYear', 'deathYear', 'birthYear', 'primaryProfession', 'knownForTitles', 'deathYear',
       'primaryProfession', 'knownForTitles'])

In [ ]:
imdb_mas50.columns

Creamos dummies para cada uno de los directores asi los podremos utilizar en los modelos:

In [ ]:

one_hot = pd.get_dummies(imdb_mas50['nconst'])

In [ ]:
one_hot.shape

Ahora creamos el ultimo dataframe antes de poder comenzar a modelar:

In [ ]:

data_df = imdb_mas50.drop('nconst',axis = 1)

In [ ]:

data_df = data_df.join(one_hot)
data_df 

In [ ]:
data_df.isnull().sum()

In [ ]:
data_df.columns

Definimos las variables dependientes e independientes. Si bien en principio se intento agregar mas variables como escritores, actores y actrices, la gran cantidad de variables que habia provoco errores de falta de memoria, por lo cual debimos reducir el numero de variables independientes:

In [ ]:
X = data_df.drop(["Clasificacion","averageRating","primaryName"], axis=1)
y = data_df["Clasificacion"]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=0)

Comenzamos utilizando un modelo de Kneighbors:

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train);

In [ ]:
y_pred = knn.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion = confusion_matrix(y_test, y_pred)
print('Confusion Matrix\n')
print(confusion)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

print('Weighted Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='weighted')))
print('Weighted Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='weighted')))
print('Weighted F1-score: {:.2f}'.format(f1_score(y_test, y_pred, average='weighted')))

from sklearn.metrics import classification_report
print('\nClassification Report\n')
print(classification_report(y_test, y_pred, target_names=['Class 1', 'Class 2']))


Continuamos con una regresion logistica:

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(max_iter=2000)
lr.fit(X_train_std, y_train)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
accuracy_score(y_test, lr.predict(X_test))

In [ ]:
from sklearn.metrics import confusion_matrix
confusion = confusion_matrix(y_test, lr.predict(X_test))
print('Confusion Matrix\n')
print(confusion)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

print('Weighted Precision: {:.2f}'.format(precision_score(y_test, lr.predict(X_test), average='weighted')))
print('Weighted Recall: {:.2f}'.format(recall_score(y_test, lr.predict(X_test), average='weighted')))
print('Weighted F1-score: {:.2f}'.format(f1_score(y_test, lr.predict(X_test), average='weighted')))

from sklearn.metrics import classification_report
print('\nClassification Report\n')
print(classification_report(y_test, lr.predict(X_test), target_names=['Class 1', 'Class 2']))

Utilizamos Grid search para evaluar mas resultados:

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

folds=StratifiedKFold(n_splits=5,shuffle=True, random_state=42)

In [ ]:
pasos = [('scaler', StandardScaler()), ('knn', KNeighborsClassifier())]

In [ ]:
pipe = Pipeline(pasos)

In [ ]:
param_grid = {'knn__n_neighbors':range(2,20,2),'knn__weights':['uniform','distance']}

In [ ]:
grid = GridSearchCV(pipe, param_grid, cv=folds)
grid.fit(X_train, y_train)

In [ ]:
##########pipe.fit(X_train,y_train)

In [ ]:
grid.score(X_test,y_test)

In [ ]:
accuracy_score(grid.predict(X_test),y_test)

Ahora analizaremos el clustering pero droppeando las variables respectivas de los directores, debido a problemas de procesamiento y de memoria:

In [ ]:
from sklearn.metrics import confusion_matrix
confusion = confusion_matrix(y_test, grid.predict(X_test))
print('Confusion Matrix\n')
print(confusion)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

print('Weighted Precision: {:.2f}'.format(precision_score(y_test, grid.predict(X_test), average='weighted')))
print('Weighted Recall: {:.2f}'.format(recall_score(y_test, grid.predict(X_test), average='weighted')))
print('Weighted F1-score: {:.2f}'.format(f1_score(y_test, grid.predict(X_test), average='weighted')))

from sklearn.metrics import classification_report
print('\nClassification Report\n')
print(classification_report(y_test, grid.predict(X_test), target_names=['Class 1', 'Class 2']))

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [ ]:
scaler=StandardScaler()
X=scaler.fit_transform(imdb_mas50.drop(["Clasificacion","averageRating","primaryName"], axis=1))
 

In [ ]:
inertia=[]
sil=[]
ch_scores=[]
k_values=range(2,40,2)

for k in k_values:
    print(k)
    km=KMeans(n_clusters=k)
    km.fit(X)
    inertia.append(km.inertia_) 

In [ ]:
plt.plot(k_values,inertia);
plt.vlines(16,np.min(inertia),np.max(inertia),linestyle='dashed');
plt.xlabel('Numero de clusters (k)',fontsize=15);plt.ylabel('Inertia',fontsize='15')

In [ ]:
k=16;
km=KMeans(n_clusters=k,random_state=0)
km.fit(X)

pca=PCA(n_components=2)
X_transformed=pca.fit_transform(X)
#plt.figure(figsize=(250,100))
sns.scatterplot(X_transformed[:,0],X_transformed[:,1],hue=km.labels_,palette='Dark2')
fig = plt.gcf()
fig.set_size_inches( 35, 35);

Generamos el grafico de clusters y vemos que se puede visualizar verticalmente distintos clusters, aunque es probable que utilizando una mayor cantidad de variables independientes estos clusters se vean mas definidos:

In [ ]:
k=16;
km=KMeans(n_clusters=k,random_state=0)
km.fit(X)

pca=PCA(n_components=2)
X_transformed=pca.fit_transform(X)
#plt.figure(figsize=(250,100))
sns.scatterplot(X_transformed[:,0],X_transformed[:,1],hue=km.labels_,palette='Dark2')

Finalmente, realizamos un grafico de barras que muestra cual es el promedio del rating para cada genero:

In [ ]:
imdb.genres.value_counts().head(10)

In [ ]:
imdb.genres.value_counts().head(10).index

In [ ]:
mask_genres = [(imdb["genres"] == "Drama") | (imdb["genres"] == "Documentary")  | (imdb.genres == 'Comedy') | (imdb.genres == 'Comedy,Drama') | (imdb.genres == 'Drama,Romance') | (imdb["genres"] == 'Horror') | (imdb.genres == 'Comedy,Romance') | (imdb.genres == 'Comedy,Drama,Romance') | (imdb.genres == 'Thriller') | (imdb.genres == 'Action')]

In [ ]:
diezprimerosgeneros = imdb.genres.value_counts().head(10).index

In [ ]:
mask_drama = imdb["genres"] == "Drama"
prom_1 = imdb[mask_drama].averageRating.mean()

In [ ]:
mask_Documentary = imdb["genres"] == "Documentary"
prom_2 = imdb[mask_Documentary].averageRating.mean()

In [ ]:
mask_Comedy = imdb["genres"] == "Comedy"
prom_3 = imdb[mask_Comedy].averageRating.mean()

In [ ]:
mask_Comedy_Drama = imdb["genres"] == "Comedy,Drama"
prom_4 =imdb[mask_Comedy_Drama].averageRating.mean()

In [ ]:
mask_Drama_Romance = imdb["genres"] == "Drama,Romance"
prom_5 =imdb[mask_Drama_Romance].averageRating.mean()

In [ ]:
mask_Horror = imdb["genres"] == "Horror"
prom_6 =imdb[mask_Horror].averageRating.mean()

In [ ]:
mask_Comedy_Romance = imdb["genres"] == "Comedy,Romance"
prom_7 =imdb[mask_Comedy_Romance].averageRating.mean()

In [ ]:
mask_Comedy_Drama_Romance = imdb["genres"] == "Comedy,Drama,Romance"
prom_8 =imdb[mask_Comedy_Romance].averageRating.mean()

In [ ]:
mask_Thriller = imdb["genres"] == "Thriller"
prom_9 =imdb[mask_Thriller].averageRating.mean()

In [ ]:
mask_Action = imdb["genres"] == "Action"
prom_10 =imdb[mask_Action].averageRating.mean()

In [ ]:
promedios = [prom_1, prom_2, prom_3, prom_4, prom_5, prom_6, prom_7, prom_8, prom_9, prom_10]

In [ ]:
generos = diezprimerosgeneros

plt.figure(figsize=(100, 5))

plt.subplot(131)
plt.bar(generos, promedios)
plt.show()